In [ ]:
# 📘 data_fetching.ipynb

# 1️⃣ Import libraries
import requests
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# 2️⃣ API endpoint (example)
API_URL = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/live"

headers = {
    "x-rapidapi-key": "YOUR_RAPIDAPI_KEY",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

# 3️⃣ Fetch live match data
response = requests.get(API_URL, headers=headers)
if response.status_code == 200:
    data = response.json()
    print("✅ API data fetched successfully")
else:
    print("❌ Failed to fetch API data", response.status_code)

# 4️⃣ Explore JSON
# This structure can vary — adjust based on Cricbuzz API response
import json
print(json.dumps(data, indent=4)[:1000])  # print first 1000 chars

# 5️⃣ Convert to pandas DataFrame
matches = []
for match in data.get("typeMatches", []):
    for series_match in match.get("seriesMatches", []):
        series = series_match.get("seriesAdWrapper", {})
        if "matches" in series:
            for m in series["matches"]:
                match_info = m.get("matchInfo", {})
                matches.append({
                    "match_id": match_info.get("matchId"),
                    "description": match_info.get("matchDesc"),
                    "team1": match_info.get("team1", {}).get("teamName"),
                    "team2": match_info.get("team2", {}).get("teamName"),
                    "venue": match_info.get("venueInfo", {}).get("ground"),
                    "match_date": match_info.get("startDate"),
                    "format": match_info.get("matchFormat"),
                    "status": match_info.get("status"),
                })

matches_df = pd.DataFrame(matches)
matches_df.head()

# 6️⃣ Connect to MySQL Database
user = "root"
password = "YOUR_PASSWORD"
host = "localhost"
database = "cricbuzz_db"

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# 7️⃣ Push data to MySQL table
matches_df.to_sql("matches", con=engine, if_exists="replace", index=False)
print("✅ Data inserted into 'matches' table")

# 8️⃣ Verify insertion
query = "SELECT * FROM matches LIMIT 5;"
pd.read_sql(query, con=engine)
